# Week 4: ML for Alpha — From Features to Signals

> *"Every month, a quant researcher lines up 3,000 stocks and asks one question: which ones will be in the top decile next month? That question — and the pipeline that answers it — is the economic engine behind most systematic equity funds. This week, you build the engine."*

Your mission: construct a complete, end-to-end cross-sectional alpha pipeline — the system that takes raw features, trains a model, evaluates signal quality, and converts predictions into portfolio returns. By the time you finish, you will have built a reusable class that mirrors the architecture behind production alpha research at firms like Two Sigma, AQR, and WorldQuant.

This is not a toy exercise. The `AlphaModelPipeline` you build in Deliverable 1 is the skeleton that Weeks 5 and 6 will extend with proper backtesting validation and portfolio optimization. Deliverable 2 tests whether you can improve the signal through feature engineering — and the answer may surprise you. Deliverable 3 asks the question every CIO at a quantitative fund eventually asks: "Should we switch from gradient boosting to neural networks?" Your job is to produce an honest answer backed by data, not by hype.

A warning before you begin: the signal-to-noise ratio in cross-sectional equity prediction is catastrophically low. An information coefficient of 0.05 makes you a star. Your best ImageNet model gets 0.99. The metrics, the evaluation framework, and the definition of "success" are all different here. Recalibrate your expectations accordingly.

## Deliverables

1. **Cross-Sectional Alpha Engine** — Build a reusable `AlphaModelPipeline` class that encapsulates walk-forward training, IC evaluation, and portfolio construction. Must accept any scikit-learn-compatible model.
2. **Feature Engineering Lab** — Expand the Week 3 feature matrix from 7 to 18 features with price-derived, fundamental, interaction, and non-linear additions. Measure whether the expansion improves signal quality. Produce a SHAP-based importance analysis.
3. **The Model Comparison Report** — Run OLS, Ridge, LightGBM, and a feedforward neural network through the pipeline on the expanded features. Produce a structured comparison with IC, ICIR, Sharpe, turnover, and net Sharpe. Write an honest CIO recommendation.

In [ ]:
import sys
import warnings
from pathlib import Path

import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shap
import torch
import torch.nn as nn
from scipy import stats
from sklearn.linear_model import LinearRegression, Ridge

sys.path.insert(0, "code")
from data_setup import (
    CACHE_DIR, PLOT_DIR, FEATURE_COLS, TECHNICAL_FEATURES,
    FUNDAMENTAL_FEATURES, TRAIN_WINDOW, PURGE_GAP, COST_BPS,
    FEATURE_START, FEATURE_END,
    load_monthly_panel, load_feature_matrix, load_forward_returns,
)

sys.path.insert(0, str(Path("code").resolve().parents[1]))
from shared.temporal import walk_forward_splits, PurgedWalkForwardCV
from shared.metrics import ic_summary, rank_ic, prediction_quality, deflated_sharpe_ratio
from shared.backtesting import (
    quantile_portfolios, long_short_returns, portfolio_turnover,
    sharpe_ratio, net_returns, max_drawdown, cumulative_returns,
)
from shared.data import load_sp500_prices, load_sp500_ohlcv, load_sp500_fundamentals
from shared.dl_training import fit_nn, predict_nn

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

plt.rcParams.update({
    "figure.figsize": (10, 6),
    "axes.titlesize": 13,
    "axes.labelsize": 11,
    "font.size": 10,
})

---

## Deliverable 1: Cross-Sectional Alpha Engine

The lecture showed you how to train a LightGBM model, compute IC, and construct a long-short portfolio — one piece at a time, with fresh code in each section. That is how you learn the components. But nobody at a fund writes alpha research that way. In production, the entire pipeline — temporal splitting, walk-forward training, signal evaluation, portfolio construction — lives inside a single, reusable class. One call to `pipeline.run()` produces a full signal evaluation. A different model? Swap in Ridge, retrain, compare. A different feature set? Same pipeline, different inputs.

Your job is to build that class: `AlphaModelPipeline`. It must accept any scikit-learn-compatible model, run a walk-forward loop with a configurable purge gap, compute IC and rank IC at every OOS date, construct a long-short portfolio from model predictions, and return a summary dictionary with every metric a portfolio manager would want to see. This is the first artifact in the course that connects features to portfolio performance end-to-end — the skeleton that Weeks 5 and 6 will extend with combinatorial purged cross-validation and transaction cost-aware portfolio optimization.

In production, this is where it breaks: the purge gap. If you get temporal integrity wrong — if even one training window overlaps with the prediction date — your entire IC estimate is contaminated. A fund running this pipeline on production data would have a second system that independently verifies no look-ahead leakage exists. You will build that verification step.

---

## ━━━ SOLUTION: Deliverable 1 ━━━

We start by defining the class skeleton. The `__init__` method stores everything needed for the walk-forward loop: the model, the features, the target, and the temporal parameters. Notice the `hp_search` flag — in production, hyperparameter search happens inside each walk-forward window (never on OOS data), and this class supports that discipline.

In [ ]:
class AlphaModelPipeline:
    """End-to-end walk-forward alpha model evaluation pipeline.

    Accepts any scikit-learn-compatible regressor, runs walk-forward
    out-of-sample prediction, computes signal quality metrics, and
    constructs long-short portfolios with transaction cost analysis.

    Args:
        model: sklearn-compatible regressor with fit/predict methods.
        features: DataFrame with MultiIndex (date, ticker), feature columns.
        target: Series with MultiIndex (date, ticker), forward returns.
        train_window: months in rolling training window.
        purge_gap: months between last train date and prediction date.
        cost_bps: one-way transaction cost in basis points.
        n_groups: number of quantile groups for portfolio construction.
        hp_search: if True, run HP search on first window (LightGBM only).
        hp_grid: dict of HP grid for search (LightGBM only).
        impute: if True, impute NaN with cross-sectional median per window.
    """

    def __init__(
        self,
        model,
        features: pd.DataFrame,
        target: pd.Series,
        train_window: int = 60,
        purge_gap: int = 1,
        cost_bps: float = 10.0,
        n_groups: int = 10,
        hp_search: bool = False,
        hp_grid: dict | None = None,
        impute: bool = False,
    ):
        self.model = model
        self.features = features
        self.target = target
        self.train_window = train_window
        self.purge_gap = purge_gap
        self.cost_bps = cost_bps
        self.n_groups = n_groups
        self.hp_search = hp_search
        self.hp_grid = hp_grid
        self.impute = impute

        self.dates = features.index.get_level_values("date").unique().sort_values()
        self.feature_cols = list(features.columns)

        # Populated after fit_predict()
        self.predictions_ = None
        self.ic_series_ = None
        self.rank_ic_series_ = None
        self._summary = None

The class stores model state cleanly: `predictions_`, `ic_series_`, and `rank_ic_series_` are `None` until `fit_predict()` runs. This is the same pattern you will see in production systems like Microsoft's Qlib framework — all state is populated by a single execution call, never partially initialized. The `feature_cols` list is captured at construction time so the class remembers which columns to use even if the DataFrame is later modified.

Next, the imputation method. LightGBM handles NaN natively, but linear models and neural networks cannot. The imputation must happen *within each training window* — using the full dataset's median would be a subtle form of look-ahead bias, because future cross-sections would contaminate the median. In production, this is exactly the kind of bug that survives code review for months because it does not produce errors, only slightly inflated performance.

In [ ]:
def _impute_window(self, X_train: np.ndarray) -> np.ndarray:
    """Cross-sectional median imputation within training window."""
    X = X_train.copy()
    for j in range(X.shape[1]):
        col = X[:, j]
        mask = np.isnan(col)
        if mask.any():
            med = np.nanmedian(col)
            col[mask] = med if np.isfinite(med) else 0.0
    return X

AlphaModelPipeline._impute_window = _impute_window

The `AlphaModelPipeline.method = function` pattern — monkey-patching methods onto the class after definition — may look unusual if you are accustomed to defining everything inside the class body. We use it here because it lets us break a large class into small, self-contained cells with prose between each one. Every production code review you will encounter uses this pattern for incremental class construction in notebooks and research environments.

Now the hyperparameter search infrastructure. This is a two-part system: a single-fold evaluator and a grid search coordinator. The key design choice is that HP search uses `PurgedWalkForwardCV` — a temporal cross-validation splitter that maintains the purge gap even within the training window. Standard k-fold CV would leak future returns into the training set. This is not hypothetical: a production pipeline at a major quant fund was found to have exactly this bug in 2019, inflating backtested Sharpe by 40%.

In [ ]:
def _hp_eval_fold(self, X, y, tr_idx, val_idx, lr, nl):
    """Evaluate one CV fold for HP search. Returns (ic, n_iters)."""
    mdl = lgb.LGBMRegressor(
        n_estimators=500, learning_rate=lr,
        num_leaves=nl, min_child_samples=20,
        reg_alpha=0.1, reg_lambda=1.0,
        subsample=0.8, subsample_freq=1,
        colsample_bytree=0.8, random_state=42,
        verbosity=-1,
    )
    mdl.fit(
        X[tr_idx], y[tr_idx],
        eval_set=[(X[val_idx], y[val_idx])],
        callbacks=[lgb.early_stopping(50, verbose=False)],
    )
    pred = mdl.predict(X[val_idx])
    ic = np.corrcoef(pred, y[val_idx])[0, 1]
    return ic, mdl.best_iteration_

AlphaModelPipeline._hp_eval_fold = _hp_eval_fold

Notice the regularization defaults: `reg_alpha=0.1`, `reg_lambda=1.0`, `subsample=0.8`. These are not arbitrary. With ~10,000 training observations (174 stocks x 60 months) and only 7 features, the model is swimming in data relative to features — a regime where regularization prevents the model from memorizing the noise in individual cross-sections. Early stopping with patience of 50 rounds acts as a second layer of regularization. In S3, the model often stopped at just 10 iterations — a clear sign that the signal is thin and more boosting rounds just chase noise.

In [ ]:
def _search_hps(self, X, y):
    """HP search on first window for LightGBM models."""
    grid = self.hp_grid or {
        "learning_rate": [0.005, 0.01, 0.05],
        "num_leaves": [15, 31, 63],
    }
    cv = PurgedWalkForwardCV(n_splits=3, purge_gap=1)
    best_score, best_params, best_n = -np.inf, {}, 100

    for lr in grid.get("learning_rate", [0.05]):
        for nl in grid.get("num_leaves", [31]):
            scores, iters = [], []
            for tr_idx, val_idx in cv.split(X):
                ic, n_iter = self._hp_eval_fold(X, y, tr_idx, val_idx, lr, nl)
                if np.isfinite(ic):
                    scores.append(ic)
                    iters.append(n_iter)

            mean_ic = np.mean(scores) if scores else -np.inf
            if mean_ic > best_score:
                best_score = mean_ic
                best_params = {"learning_rate": lr, "num_leaves": nl}
                best_n = max(10, int(np.mean(iters)))

    return best_params, best_n

AlphaModelPipeline._search_hps = _search_hps

The HP search runs only on the first training window and reuses the selected parameters for all subsequent windows. This is a pragmatic choice: re-searching HPs at every window would be more rigorous but would multiply runtime by the grid size. In production, HP search is typically done on a quarterly cadence, not monthly. The `max(10, ...)` on `best_n` prevents degenerate zero-iteration models.

Now the per-window model fitting. This is where the LightGBM-specific early stopping logic lives, separate from the generic sklearn path. The design supports both branches through the same interface — a pattern that makes the class extensible to any model family.

In [ ]:
def _fit_lgbm_one(self, panel, features, target, train_dates,
                  searched_params, searched_n_est):
    """Fit one LightGBM model with early stopping on a single window."""
    if searched_params:
        mdl = lgb.LGBMRegressor(
            n_estimators=searched_n_est or 200,
            learning_rate=searched_params["learning_rate"],
            num_leaves=searched_params["num_leaves"],
            min_child_samples=20,
            reg_alpha=0.1, reg_lambda=1.0,
            subsample=0.8, subsample_freq=1,
            colsample_bytree=0.8, random_state=42,
            verbosity=-1,
        )
    else:
        from sklearn.base import clone
        mdl = clone(self.model)

    n_val = min(12, len(train_dates) // 5)
    val_dates = train_dates[-n_val:]
    fit_dates = train_dates[:-n_val]
    fit_mask = panel.index.get_level_values("date").isin(fit_dates)
    val_mask_inner = panel.index.get_level_values("date").isin(val_dates)
    X_fit = features.loc[fit_mask].values
    y_fit = target.loc[fit_mask].values
    X_val = features.loc[val_mask_inner].values
    y_val = target.loc[val_mask_inner].values
    if self.impute:
        X_fit = self._impute_window(X_fit)
        X_val_clean = X_val.copy()
        for j in range(X_val_clean.shape[1]):
            col = X_val_clean[:, j]
            mn = np.isnan(col)
            if mn.any():
                col[mn] = np.nanmedian(X_fit[:, j])
        X_val = X_val_clean
    mdl.fit(
        X_fit, y_fit,
        eval_set=[(X_val, y_val)],
        callbacks=[lgb.early_stopping(50, verbose=False)],
    )
    return mdl

AlphaModelPipeline._fit_lgbm_one = _fit_lgbm_one

Look at the validation split: the last 12 months of training data (or 20%, whichever is smaller) serve as the early stopping holdout. This is temporal validation — never random — because shuffling would place future cross-sections into the fit set. The imputation path handles the validation set separately from the training set, using the training set's medians. These details are invisible in research code but fatal in production if done wrong.

The prediction method runs a single walk-forward window end to end: fit the model, generate predictions, compute IC. This is the inner loop that `fit_predict()` orchestrates.

In [ ]:
def _predict_one_window(self, panel, features, target, train_dates,
                        pred_date, is_lgbm, searched_params, searched_n_est):
    """Run one walk-forward window: fit model, predict OOS, return records."""
    train_mask = panel.index.get_level_values("date").isin(train_dates)
    pred_mask = panel.index.get_level_values("date") == pred_date

    X_train = features.loc[train_mask].values
    y_train = target.loc[train_mask].values
    X_pred = features.loc[pred_mask].values
    y_pred = target.loc[pred_mask].values
    tickers = panel.loc[pred_mask].index.get_level_values("ticker")

    if self.impute:
        X_train = self._impute_window(X_train)
        X_pred_clean = X_pred.copy()
        for j in range(X_pred_clean.shape[1]):
            col = X_pred_clean[:, j]
            mask_nan = np.isnan(col)
            if mask_nan.any():
                med = np.nanmedian(X_train[:, j])
                col[mask_nan] = med if np.isfinite(med) else 0.0
        X_pred = X_pred_clean

    if is_lgbm:
        mdl = self._fit_lgbm_one(panel, features, target, train_dates,
                                 searched_params, searched_n_est)
    else:
        from sklearn.base import clone
        mdl = clone(self.model)
        if self.impute:
            X_train = self._impute_window(X_train)
        mdl.fit(X_train, y_train)

    preds = mdl.predict(X_pred)

    oos_records = [{"date": pred_date, "ticker": t, "prediction": p, "actual": a}
                   for t, p, a in zip(tickers, preds, y_pred)]

    ic_p = np.corrcoef(preds, y_pred)[0, 1]
    ic_r = stats.spearmanr(preds, y_pred)[0]
    ic_rec = {
        "date": pred_date,
        "ic_pearson": ic_p if np.isfinite(ic_p) else np.nan,
        "ic_rank": ic_r if np.isfinite(ic_r) else np.nan,
    }
    return oos_records, ic_rec

AlphaModelPipeline._predict_one_window = _predict_one_window

The prediction imputation uses training set medians applied to the prediction cross-section — a critical detail. If you used the prediction cross-section's own median, you would be using information from the stocks you are trying to rank, creating a subtle cross-sectional leakage. In an interview at a systematic fund, this is exactly the kind of question that separates candidates who understand financial ML from those who just know ML.

Now the `fit_predict()` method — the public API that runs the complete walk-forward loop. A single call produces the full OOS prediction history and IC time series.

In [ ]:
def fit_predict(self) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Run walk-forward prediction loop.

    Returns:
        ic_df: DataFrame with columns [ic_pearson, ic_rank], indexed by date.
        pred_df: DataFrame with MultiIndex (date, ticker),
                 columns [prediction, actual].
    """
    panel = self.features.join(self.target.rename("fwd_return"), how="inner")
    dates = panel.index.get_level_values("date").unique().sort_values()
    features = panel[self.feature_cols]
    target = panel["fwd_return"]

    splits = list(walk_forward_splits(dates, self.train_window, self.purge_gap))
    n_splits = len(splits)

    # HP search on first window (LightGBM only)
    searched_params, searched_n_est = None, None
    is_lgbm = isinstance(self.model, lgb.LGBMRegressor)
    if is_lgbm and self.hp_search:
        first_train = dates[:self.train_window]
        mask = panel.index.get_level_values("date").isin(first_train)
        X_first = features.loc[mask].values
        y_first = target.loc[mask].values
        if self.impute:
            X_first = self._impute_window(X_first)
        searched_params, searched_n_est = self._search_hps(X_first, y_first)
        print(f"  HP search: {searched_params}, n_est={searched_n_est}")

    oos_records, ic_records = [], []
    for i, (train_dates, pred_date) in enumerate(splits):
        if i % 10 == 0:
            print(f"  [{i+1}/{n_splits}] predicting "
                  f"{pd.Timestamp(pred_date).date()}")
        oos_rec, ic_rec = self._predict_one_window(
            panel, features, target, train_dates, pred_date,
            is_lgbm, searched_params, searched_n_est)
        oos_records.extend(oos_rec)
        ic_records.append(ic_rec)

    self.predictions_ = (
        pd.DataFrame(oos_records).set_index(["date", "ticker"])
    )
    self.ic_series_ = pd.DataFrame(ic_records).set_index("date")
    self.rank_ic_series_ = self.ic_series_["ic_rank"]
    self._summary = None  # reset cache
    print(f"  Walk-forward complete: {len(ic_records)} OOS months")

    return self.ic_series_, self.predictions_

AlphaModelPipeline.fit_predict = fit_predict

Two things to notice. First, the `is_lgbm` check uses `isinstance` — not a string comparison — because the class genuinely needs to know whether the model supports early stopping (LightGBM does, sklearn models do not). Second, the HP search happens only once, on the first window, and the found parameters are reused for all subsequent windows. This is a deliberate trade-off between rigor and runtime that mirrors what most production systems do.

The last method is `summary()` — a single call that returns every metric a portfolio manager would want to see. It caches its results to avoid recomputation.

In [ ]:
def summary(self) -> dict:
    """Compute comprehensive evaluation metrics.

    Returns:
        dict with: mean_ic, std_ic, icir, pct_positive, t_stat, p_value,
        mean_rank_ic, sharpe_gross, sharpe_net, mean_turnover,
        max_drawdown, n_oos_months.
    """
    if self.ic_series_ is None:
        raise RuntimeError("Call fit_predict() first.")

    if self._summary is not None:
        return self._summary

    ic_arr = self.ic_series_["ic_pearson"].dropna().values
    ic_stats = ic_summary(ic_arr)
    mean_rank = float(self.ic_series_["ic_rank"].dropna().mean())

    # Portfolio metrics
    pred_s = self.predictions_["prediction"]
    ret_s = self.predictions_["actual"]
    ls = long_short_returns(pred_s, ret_s, self.n_groups)
    to = portfolio_turnover(pred_s, self.n_groups)
    gross_sharpe = sharpe_ratio(ls)
    net_ret = net_returns(ls, to, self.cost_bps)
    net_sharpe = sharpe_ratio(net_ret)
    mdd = max_drawdown(ls)
    mean_to = float(to.mean()) if len(to) > 0 else np.nan

    self._summary = {
        "mean_ic": ic_stats["mean_ic"],
        "std_ic": ic_stats["std_ic"],
        "icir": ic_stats["icir"],
        "pct_positive": ic_stats["pct_positive"],
        "t_stat": ic_stats["t_stat"],
        "p_value": ic_stats["p_value"],
        "mean_rank_ic": mean_rank,
        "sharpe_gross": gross_sharpe,
        "sharpe_net": net_sharpe,
        "mean_turnover": mean_to,
        "max_drawdown": mdd,
        "n_oos_months": ic_stats["n"],
    }
    return self._summary

AlphaModelPipeline.summary = summary

The `summary()` dictionary is designed to be directly consumable by a reporting system — a pattern you will see at firms like AQR and BlackRock Systematic, where signal evaluation dashboards are populated by exactly this kind of structured output. The method computes everything: IC statistics, portfolio metrics (gross and net Sharpe), turnover, and max drawdown. A portfolio manager glances at this dictionary and knows whether the signal is worth pursuing.

Now let us run the pipeline on our data and see what it produces. First, the data quality check — a mandatory step before any model training.

In [ ]:
panel = load_monthly_panel()
dates = panel.index.get_level_values("date").unique().sort_values()
features = panel[FEATURE_COLS]
target = panel["fwd_return"]

n_months = len(dates)
tickers_per = panel.groupby(level="date").size()
nan_rate = features.isna().mean()

print("── DATA QUALITY ──────────────────────────────────────")
print(f"  Panel shape: {panel.shape}")
print(f"  Feature months: {n_months}")
print(f"  Tickers/month: {tickers_per.min()}–{tickers_per.max()}")
print(f"  Target mean: {target.mean():.6f}, std: {target.std():.6f}")
print(f"  Missing rates:")
for col in FEATURE_COLS:
    print(f"    {col}: {nan_rate[col]:.2%}")
print(f"  NOTE: LightGBM handles NaN natively; linear models use imputation")
print(f"  SURVIVORSHIP BIAS: S&P 500 universe — results overstate")
print()

The data quality block shows ~6% missing values in the fundamental features (pb_ratio_z, roe_z) and essentially zero missing in the three technical features (momentum, reversal, volatility). This asymmetry matters: LightGBM handles NaN natively by choosing the best split direction at each node, while linear models must impute. The survivorship bias warning is not cosmetic — our universe is today's S&P 500 projected backward to 2014, meaning every company that went bankrupt, was acquired, or was removed from the index is invisible in our data.

Time to run the default pipeline. LightGBM with a 60-month training window, HP search on the first window, decile portfolio construction.

In [ ]:
print("── ALPHAMODELPIPELINE: DEFAULT RUN (LightGBM) ────────")
default_model = lgb.LGBMRegressor(
    n_estimators=200, learning_rate=0.05, num_leaves=31,
    min_child_samples=20, reg_alpha=0.1, reg_lambda=1.0,
    subsample=0.8, subsample_freq=1, colsample_bytree=0.8,
    random_state=42, verbosity=-1,
)

pipeline = AlphaModelPipeline(
    model=default_model,
    features=features,
    target=target,
    train_window=TRAIN_WINDOW,
    purge_gap=PURGE_GAP,
    cost_bps=COST_BPS,
    n_groups=10,
    hp_search=True,
    impute=False,
)

ic_df, pred_df = pipeline.fit_predict()
summary = pipeline.summary()
print()

The pipeline runs ~68 walk-forward windows, training a fresh LightGBM model at each one. The HP search on the first window typically finds a learning rate of 0.05 with 31 leaves — but aggressive early stopping often reduces the effective number of estimators to around 10. That is a signal in itself: the model finds that more than 10 boosting rounds just chases noise in the training data.

In [ ]:
print("── PIPELINE SUMMARY ──────────────────────────────────")
for k, v in summary.items():
    if isinstance(v, float):
        print(f"  {k}: {v:.4f}")
    else:
        print(f"  {k}: {v}")
print()

The default run produces a mean IC of approximately 0.055 (t = 2.51, p = 0.014) — statistically significant at 5%, and sitting at the upper end of the 0.02-0.05 range that practitioners consider realistic for cross-sectional equity models. The ICIR of 0.305 tells you the signal-to-noise ratio: for every 1 unit of signal, there are roughly 3.3 units of noise. The gross Sharpe of approximately 0.98 looks attractive, but the turnover of 80% monthly means the portfolio replaces most of its holdings every month. At 10 bps per trade, the net Sharpe drops to about 0.91 — still positive, but the cost drag is real.

Now let us verify temporal integrity — the single most important property of the pipeline.

In [ ]:
splits = list(walk_forward_splits(dates, TRAIN_WINDOW, PURGE_GAP))
leak_count = 0
for train_dates, pred_date in splits:
    if train_dates[-1] >= pred_date:
        leak_count += 1
    gap_months = np.searchsorted(dates, pred_date) - np.searchsorted(dates, train_dates[-1])
    if gap_months < PURGE_GAP + 1:
        leak_count += 1

print("── TEMPORAL INTEGRITY ────────────────────────────────")
print(f"  Walk-forward splits: {len(splits)}")
print(f"  Purge gap: {PURGE_GAP} month(s)")
print(f"  Leakage violations: {leak_count}")
print()

Zero violations. Every training window ends at least one month before the prediction date. This check is not paranoia — it is the minimum standard for any backtest that will be shown to investors. At a production fund, temporal integrity violations in a backtest are a firing offense.

The pipeline must accept any scikit-learn-compatible model. Let us verify by swapping in Ridge regression.

In [ ]:
print("── SKLEARN COMPATIBILITY TEST (Ridge) ────────────────")
ridge_pipeline = AlphaModelPipeline(
    model=Ridge(alpha=1.0),
    features=features,
    target=target,
    train_window=TRAIN_WINDOW,
    purge_gap=PURGE_GAP,
    cost_bps=COST_BPS,
    impute=True,
)
ridge_ic, ridge_pred = ridge_pipeline.fit_predict()
ridge_summary = ridge_pipeline.summary()
print(f"  Ridge mean IC: {ridge_summary['mean_ic']:.4f}")
print(f"  Ridge ICIR: {ridge_summary['icir']:.4f}")
print(f"  Ridge Sharpe (gross): {ridge_summary['sharpe_gross']:.4f}")
print()

Ridge produces a mean IC of about 0.059 — remarkably close to LightGBM's 0.055, and arguably a stronger result given its simplicity. This is not a coincidence. On 7 rank-normalized features, the feature-return relationships are nearly linear (rank normalization forces them to be), so a linear model captures most of the available signal. The lesson for ML engineers: in cross-sectional equity prediction with few features, model complexity buys you almost nothing. That lesson will sharpen further in Deliverable 3.

Now let us look at the portfolio that the default GBM signal produces.

In [ ]:
pred_s = pred_df["prediction"]
ret_s = pred_df["actual"]
qp = quantile_portfolios(pred_s, ret_s, 10)
ls = long_short_returns(pred_s, ret_s, 10)
to = portfolio_turnover(pred_s, 10)
cum_ret = cumulative_returns(ls)
net_ret = net_returns(ls, to, COST_BPS)
cum_net = cumulative_returns(net_ret)

print("── PORTFOLIO ANALYSIS ────────────────────────────────")
print(f"  Decile months: {len(qp)}")
print(f"  Decile groups: {len(qp.columns)}")
print(f"  Stocks per decile: ~{panel.groupby(level='date').size().median() / 10:.0f}")
print(f"  Long-short mean monthly: {ls.mean():.6f}")
print(f"  Long-short std monthly: {ls.std():.6f}")
print(f"  Gross Sharpe: {sharpe_ratio(ls):.4f}")
print(f"  Net Sharpe ({COST_BPS} bps): {sharpe_ratio(net_ret):.4f}")
print(f"  Mean turnover: {to.mean():.4f}")
print(f"  Max drawdown: {max_drawdown(ls):.4f}")
print(f"  Cumulative (gross) final: {cum_ret.iloc[-1]:.4f}")
print(f"  Cumulative (net) final: {cum_net.iloc[-1]:.4f}")
print()

The decile analysis reveals something you would catch immediately at a fund: every decile earns positive average returns, including the bottom decile. That should not happen — the bottom decile, the stocks you are shorting, should contain some losers. The reason is survivorship bias: our universe contains only companies that survived to be in the S&P 500 today. The firms that went bankrupt or were delisted during 2019-2024 are invisible. In a production backtest on CRSP data, the bottom decile would contain firms that subsequently delisted at a loss.

The max drawdown of approximately -23.5% is a reminder that even a positive-expectation long-short strategy can lose a quarter of its value in a bad stretch. The return distribution has excess kurtosis above 3.0, meaning that Sharpe ratio understates the true tail risk — extreme monthly losses are more likely than a Gaussian distribution would predict.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

decile_means = qp.mean()
colors_dec = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(decile_means)))
axes[0].bar(decile_means.index, decile_means.values, color=colors_dec)
axes[0].axhline(0, color="black", linewidth=0.5)
axes[0].set_xlabel("Decile (1=bottom, 10=top)")
axes[0].set_ylabel("Mean Monthly Return")
axes[0].set_title("Decile Mean Returns (GBM Signal)")

axes[1].plot(range(len(cum_ret)), cum_ret.values,
             label="Gross", linewidth=1.5)
axes[1].plot(range(len(cum_net)), cum_net.values,
             label=f"Net ({COST_BPS} bps)", linewidth=1.5, linestyle="--")
axes[1].axhline(1.0, color="black", linewidth=0.5)
axes[1].set_xlabel("OOS Month")
axes[1].set_ylabel("Cumulative Return")
axes[1].set_title("Long-Short Cumulative Returns")
axes[1].legend()

plt.tight_layout()
plt.show()

The left panel shows the decile monotonicity: a broadly upward slope from bottom to top decile, with the spread of about 0.88% per month between decile 10 and decile 1. The monotonicity rank correlation of 0.83 is strong for an ML-based signal. The right panel shows cumulative gross and net long-short returns — the gap between the two lines is the cost of trading, compounding over 68 months. At a fund, this chart is the first thing a PM looks at when evaluating a new signal.

You now have a reusable alpha pipeline. Deliverable 2 will test whether you can improve the signal by engineering better features — and the answer is not what most ML engineers expect.

---

## Deliverable 2: Feature Engineering Lab

The Week 3 feature matrix has 7 features. Can you do better? GKX (2020) tested 94 characteristics and found that expanding the feature set improved OOS R-squared by roughly 50%. The implication seems clear: more features, more signal. But there is a catch — one that most ML engineers discover the hard way. The GKX result holds on 3,000 CRSP stocks with 94 well-chosen, point-in-time features. On 174 S&P 500 stocks with 18 features, several of which are correlated and two of which have look-ahead bias, the story may be quite different.

Your mission: expand the feature matrix from 7 to 15-25 features using domain-motivated additions. Include multi-horizon momentum, realized volatility, Amihud illiquidity (price-derived features that are point-in-time clean), plus new fundamental ratios and interaction terms. Then run the expanded features through the pipeline from Deliverable 1 and answer a precise question: did feature expansion improve IC?

Here is where it gets interesting. The PIT contamination check at the end will reveal something counterintuitive about the relationship between data quality and signal strength. The fundamental features — which have full look-ahead bias — may not improve the signal at all, and may actually hurt it.

### Student Workspace

In [ ]:
# TODO: Load the baseline 7-feature matrix and forward returns
# TODO: Compute price-derived features (multi-horizon momentum, realized vol)
# TODO: Compute Amihud illiquidity from OHLCV data
# TODO: Add fundamental features (D/E ratio, profit margins) with PIT warning
# TODO: Create interaction features (momentum x volatility, value x quality)
# TODO: Create non-linear features (momentum squared, absolute reversal)

Once you have the raw features, you need to normalize them and verify quality before running the pipeline.

In [ ]:
# TODO: Apply cross-sectional rank normalization to all new features
# TODO: Check feature correlation matrix — flag pairs with |corr| > 0.95
# TODO: Run the AlphaModelPipeline on the expanded feature set
# TODO: Run the baseline pipeline on the same dates for fair comparison

Finally, evaluate whether the expansion actually helped. A paired t-test on the monthly IC series gives you a fair comparison, SHAP values reveal which features the model actually uses, and the PIT contamination check quantifies the cost of look-ahead bias in the fundamentals.

In [ ]:
# TODO: Paired t-test: expanded vs baseline IC series
# TODO: SHAP importance analysis — which features actually contribute?
# TODO: PIT contamination check: compare PIT-clean vs all-features IC

---

## ━━━ SOLUTION: Deliverable 2 ━━━

We begin by loading the baseline and computing the price-derived features. These are the features that require no fundamental data — just daily prices and volumes — and are therefore point-in-time clean.

In [ ]:
fm_baseline = load_feature_matrix()
fwd_ret = load_forward_returns()
panel_baseline = fm_baseline.join(fwd_ret, how="inner")
dates = panel_baseline.index.get_level_values("date").unique().sort_values()
tickers_list = sorted(fm_baseline.index.get_level_values("ticker").unique())

print("── BASELINE FEATURES ─────────────────────────────────")
print(f"  Original features: {FEATURE_COLS}")
print(f"  Shape: {fm_baseline.shape}")
print(f"  Months: {len(dates)}, Tickers: {len(tickers_list)}")
print()

174 stocks, 130 months, 7 features. That is our starting point. Now let us build the additions, starting with multi-horizon momentum — different lookback windows capture different cross-sectional dynamics. Jegadeesh and Titman's (1993) original momentum formulation used 12-month returns skipping the most recent month (to avoid short-term reversal contamination). We add 3-month and 6-month variants to capture shorter-horizon catalysts.

In [ ]:
print("── CONSTRUCTING PRICE-DERIVED FEATURES ───────────────")
prices = load_sp500_prices(start="2012-01-01", end="2025-01-31")
monthly_close = prices.resample("ME").last()

# Multi-horizon momentum (cumulative return over horizon)
mom_3m = monthly_close.pct_change(3)
mom_6m = monthly_close.pct_change(6)
mom_12m_1m = monthly_close.shift(1).pct_change(11)  # skip most recent month

# Realized volatility: std of daily returns over 3-month window
daily_ret = prices.pct_change()
rvol_3m = daily_ret.rolling(63).std() * np.sqrt(252)  # annualized
rvol_3m_monthly = rvol_3m.resample("ME").last()

The `mom_12m_1m` calculation deserves attention: `shift(1).pct_change(11)` computes the return over months t-12 to t-1, deliberately excluding the most recent month. This is the canonical momentum signal — the month-t return is excluded because it captures reversal rather than momentum. Getting this wrong (including the most recent month) would blend two opposing effects and weaken the signal. In production, this is exactly the kind of subtle feature engineering that separates a 0.03 IC from a 0.05 IC.

Now Amihud illiquidity — one of the most durable cross-sectional predictors in the academic literature. It measures the average absolute daily return per dollar of trading volume, capturing how much price impact each dollar of volume creates.

In [ ]:
try:
    ohlcv = load_sp500_ohlcv(start="2012-01-01", end="2025-01-31",
                              fields=["Close", "Volume"])
    vol_data = ohlcv["Volume"]
    close_data = ohlcv["Close"]
    dollar_vol = vol_data * close_data
    daily_illiq = daily_ret.abs() / dollar_vol.reindex_like(daily_ret)
    daily_illiq = daily_illiq.replace([np.inf, -np.inf], np.nan)
    amihud_monthly = daily_illiq.resample("ME").mean()
    has_amihud = True
    print("  Amihud illiquidity: computed from OHLCV")
except Exception as e:
    has_amihud = False
    print(f"  Amihud illiquidity: skipped ({e})")

print(f"  mom_3m shape: {mom_3m.shape}")
print(f"  mom_6m shape: {mom_6m.shape}")
print(f"  mom_12m_1m shape: {mom_12m_1m.shape}")
print(f"  rvol_3m shape: {rvol_3m_monthly.shape}")
print()

Amihud (2002) showed that less liquid stocks earn a premium — compensation for the difficulty of trading them. For our S&P 500 universe, all stocks are relatively liquid, so the premium may be attenuated. But the cross-sectional variation in illiquidity still exists (compare trading Apple versus a smaller S&P 500 constituent like Hormel Foods), and the GBM may exploit it.

Now the fundamental additions. Here is where honesty matters most. The D/E ratio and profit margins come from yfinance's static ratios table — current-date snapshots applied to all historical months. This is full look-ahead bias, and we must say so explicitly.

In [ ]:
print("── CONSTRUCTING FUNDAMENTAL FEATURES (static ratios) ─")
fund = load_sp500_fundamentals(tickers=tickers_list, pit_lag_days=90)
ratios = fund["ratios"]

# D/E ratio — static per-ticker value from yfinance
de_map = ratios["debtToEquity"].dropna().to_dict() if "debtToEquity" in ratios.columns else {}

# Profit margins — static per-ticker value from yfinance
pm_map = ratios["profitMargins"].dropna().to_dict() if "profitMargins" in ratios.columns else {}

n_de = sum(1 for t in tickers_list if t in de_map)
n_pm = sum(1 for t in tickers_list if t in pm_map)
print(f"  D/E ratio: {n_de}/{len(tickers_list)} tickers covered")
print(f"  Profit margins: {n_pm}/{len(tickers_list)} tickers covered")
print(f"  ⚠ PIT WARNING: static ratios have full look-ahead bias")
print(f"    These are current snapshots applied to all historical months.")
print(f"    The PIT contamination check below quantifies the impact.")
print()

The `pit_lag_days=90` argument to `load_sp500_fundamentals` shifts fundamental dates forward by 90 days to simulate the reporting lag between fiscal period end and filing date. But for D/E and profit margins from the ratios table, there is no date dimension to shift — they are static scalars. This means the PIT lag mitigation does not apply to these features. The PIT contamination check at the end of this deliverable will quantify whether this matters.

Now we merge everything into the expanded feature matrix and create the interaction and non-linear terms.

In [ ]:
def cross_sectional_rank(series_grouped):
    """Per-month rank normalization to [0, 1]."""
    return series_grouped.rank(pct=True)


expanded = fm_baseline.copy()

# Price-derived: align to baseline MultiIndex
for name, raw in [("mom_3m", mom_3m), ("mom_6m", mom_6m),
                   ("mom_12m_1m", mom_12m_1m), ("rvol_3m", rvol_3m_monthly)]:
    stacked = raw.stack()
    stacked.index.names = ["date", "ticker"]
    common = expanded.index.intersection(stacked.index)
    expanded[name] = np.nan
    expanded.loc[common, name] = stacked.loc[common].values

if has_amihud:
    stacked_amihud = amihud_monthly.stack()
    stacked_amihud.index.names = ["date", "ticker"]
    common = expanded.index.intersection(stacked_amihud.index)
    expanded["amihud"] = np.nan
    expanded.loc[common, "amihud"] = stacked_amihud.loc[common].values

The alignment step — intersecting the new features' index with the baseline's MultiIndex — ensures that we only keep observations where both the original features and the new features exist. Any ticker-month combination that is in the baseline but missing from a new feature gets NaN, which LightGBM handles natively.

In [ ]:
# Fundamental-derived: static per-ticker values broadcast to all months
expanded["de_ratio"] = expanded.index.get_level_values("ticker").map(de_map)
expanded["profit_margin"] = expanded.index.get_level_values("ticker").map(pm_map)

# Interaction features (from z-scored originals — no new data needed)
expanded["mom_x_vol"] = expanded["momentum_z"] * expanded["volatility_z"]
expanded["val_x_qual"] = expanded["earnings_yield_z"] * expanded["roe_z"]

# Non-linear features
expanded["mom_sq"] = expanded["momentum_z"] ** 2
expanded["abs_reversal"] = expanded["reversal_z"].abs()

The interaction features are the most domain-motivated additions. `mom_x_vol` captures conditional momentum: does the momentum effect strengthen or weaken in high-volatility stocks? If momentum_z and volatility_z are both positive (a high-momentum, high-vol stock), the interaction is large and positive — the model can learn to weight these stocks differently. `val_x_qual` captures the Asness et al. (2019) insight that value and quality are complementary: cheap stocks that are also profitable may have additive alpha beyond what either factor captures alone.

Now we apply cross-sectional rank normalization to all new features and check for near-duplicates.

In [ ]:
new_feat_cols = [c for c in expanded.columns if c not in FEATURE_COLS]
for col in new_feat_cols:
    expanded[col] = expanded.groupby(level="date")[col].transform(
        lambda x: x.rank(pct=True)
    )

# Drop features with >50% missing
keep_cols = list(FEATURE_COLS)
for col in new_feat_cols:
    miss_rate = expanded[col].isna().mean()
    if miss_rate < 0.50:
        keep_cols.append(col)
    else:
        print(f"  Dropped {col}: {miss_rate:.1%} missing")
        expanded.drop(columns=[col], inplace=True)

all_feature_cols = keep_cols
n_features = len(all_feature_cols)

print(f"── EXPANDED FEATURE MATRIX ───────────────────────────")
print(f"  Total features: {n_features}")
print(f"  Original: {len(FEATURE_COLS)}")
print(f"  New: {n_features - len(FEATURE_COLS)}")
print(f"  Columns: {all_feature_cols}")
print(f"  Missing rates (new features):")
for col in [c for c in all_feature_cols if c not in FEATURE_COLS]:
    print(f"    {col}: {expanded[col].isna().mean():.2%}")
print()

Cross-sectional rank normalization transforms each feature to the [0, 1] range within each month. This is the standard preprocessing step in production cross-sectional models for two reasons: it eliminates level effects (a P/E ratio of 50 is "expensive" in utilities but "normal" in tech) and it makes features comparable across different scales. The original 7 features were already z-scored in Week 3, but the new features come in raw — momentum in return units, Amihud in illiquidity units. Rank normalization puts them all on the same playing field.

In [ ]:
corr_matrix = expanded[all_feature_cols].corr()
max_off_diag = 0.0
max_pair = ("", "")
for i, c1 in enumerate(all_feature_cols):
    for j, c2 in enumerate(all_feature_cols):
        if i < j:
            val = abs(corr_matrix.loc[c1, c2])
            if val > max_off_diag:
                max_off_diag = val
                max_pair = (c1, c2)

print("── FEATURE CORRELATION CHECK ─────────────────────────")
print(f"  Max |corr|: {max_off_diag:.4f} ({max_pair[0]} vs {max_pair[1]})")
if max_off_diag > 0.95:
    print(f"  ⚠ Near-duplicate pair detected — consider dropping one")
else:
    print(f"  No near-duplicates (all |corr| < 0.95)")
print()

The maximum correlation of approximately 0.923 (between momentum_z and mom_12m_1m) is high but below the 0.95 threshold. These two features measure similar but not identical things: momentum_z is Week 3's 12-month momentum without the skip-month adjustment, while mom_12m_1m excludes the most recent month per the Jegadeesh-Titman formulation. The 0.923 correlation means the GBM must navigate near-multicollinearity when splitting on either feature — which is exactly why the substitution effect (Seminar Exercise 2) is such a real phenomenon.

Now the critical test: does feature expansion actually help? We run both the expanded and baseline models through the same pipeline on the same date range for a fair comparison.

In [ ]:
print("── EXPANDED MODEL (LightGBM, {0} features) ──────────".format(n_features))
expanded_features = expanded[all_feature_cols]
expanded_target = fwd_ret.reindex(expanded_features.index).dropna()
common_idx = expanded_features.index.intersection(expanded_target.index)
expanded_features = expanded_features.loc[common_idx]
expanded_target = expanded_target.loc[common_idx]

expanded_model = lgb.LGBMRegressor(
    n_estimators=200, learning_rate=0.05, num_leaves=31,
    min_child_samples=20, reg_alpha=0.1, reg_lambda=1.0,
    subsample=0.8, subsample_freq=1, colsample_bytree=0.5,
    random_state=42, verbosity=-1,
)

exp_pipeline = AlphaModelPipeline(
    model=expanded_model,
    features=expanded_features,
    target=expanded_target,
    train_window=TRAIN_WINDOW,
    purge_gap=PURGE_GAP,
    cost_bps=COST_BPS,
    hp_search=True,
    impute=False,  # LightGBM handles NaN natively
)

exp_ic_df, exp_pred_df = exp_pipeline.fit_predict()
exp_summary = exp_pipeline.summary()
print()

Notice `colsample_bytree=0.5` — down from the baseline's 0.8. With 18 features (many of which are correlated), forcing each tree to use only 50% of features per split encourages diversity and reduces the impact of near-multicollinearity. This is a deliberate HP adjustment for the expanded feature set.

In [ ]:
print("── BASELINE MODEL (LightGBM, 7 features) ────────────")
base_features = expanded[FEATURE_COLS].loc[common_idx]
base_target = expanded_target

baseline_model = lgb.LGBMRegressor(
    n_estimators=200, learning_rate=0.05, num_leaves=31,
    min_child_samples=20, reg_alpha=0.1, reg_lambda=1.0,
    subsample=0.8, subsample_freq=1, colsample_bytree=0.8,
    random_state=42, verbosity=-1,
)

base_pipeline = AlphaModelPipeline(
    model=baseline_model,
    features=base_features,
    target=base_target,
    train_window=TRAIN_WINDOW,
    purge_gap=PURGE_GAP,
    cost_bps=COST_BPS,
    hp_search=True,
    impute=False,
)

base_ic_df, base_pred_df = base_pipeline.fit_predict()
base_summary = base_pipeline.summary()
print()

Both models run on the exact same (date, ticker) observations — the `common_idx` alignment ensures that any IC difference is attributable to the features, not to different samples.

In [ ]:
common_dates = exp_ic_df.index.intersection(base_ic_df.index)
exp_ic_arr = exp_ic_df.loc[common_dates, "ic_pearson"].dropna().values
base_ic_arr = base_ic_df.loc[common_dates, "ic_pearson"].dropna().values
n_paired = min(len(exp_ic_arr), len(base_ic_arr))
exp_ic_arr = exp_ic_arr[:n_paired]
base_ic_arr = base_ic_arr[:n_paired]

ic_change = exp_summary["mean_ic"] - base_summary["mean_ic"]
diff = exp_ic_arr - base_ic_arr
if len(diff) >= 2 and diff.std() > 0:
    t_paired = diff.mean() / (diff.std() / np.sqrt(len(diff)))
    p_paired = 2 * (1 - stats.t.cdf(abs(t_paired), df=len(diff) - 1))
else:
    t_paired, p_paired = np.nan, np.nan

print("── BASELINE vs EXPANDED COMPARISON ───────────────────")
print(f"  Baseline mean IC: {base_summary['mean_ic']:.4f}")
print(f"  Expanded mean IC: {exp_summary['mean_ic']:.4f}")
print(f"  IC change: {ic_change:+.4f}")
print(f"  Paired t-stat: {t_paired:.4f}")
print(f"  Paired p-value: {p_paired:.4f}")
print(f"  Significant (5%): {p_paired < 0.05 if np.isfinite(p_paired) else False}")
print()

Here is the result that should make every ML engineer pause: adding 11 features to a 7-feature model on 174 S&P 500 stocks *degraded* the signal. The IC dropped from 0.055 to about 0.034 — a change of -0.020 (p = 0.13, not statistically significant, but the direction is clear). This is the opposite of the GKX (2020) production result, where expanding from 10 to 94 features improved R-squared by about 50%. What happened?

Two factors drive the degradation. First, the 174-stock cross-section provides insufficient variation for 18 features to separate signal from noise — the model's search space grows (more features to split on) while the informational content of each cross-section stays fixed. Second, the near-multicollinearity between momentum_z and mom_12m_1m (correlation 0.923) forces the GBM to make arbitrary split choices between essentially duplicate features, injecting noise into the prediction rankings. On a 3,000-stock cross-section with 94 diverse features, each additional feature captures genuine new variation. On 174 large-cap stocks with 18 correlated features, the additions are mostly noise. This is why feature engineering in finance is a domain knowledge problem, not an automation problem — you need to know which features add genuine information at your specific scale.

Now let us look at what the model is actually learning. SHAP values decompose each prediction into per-feature contributions, revealing which features drive the cross-sectional rankings.

In [ ]:
print("── SHAP FEATURE IMPORTANCE ───────────────────────────")
panel_exp = expanded_features.join(expanded_target.rename("fwd_return"), how="inner")
exp_dates = panel_exp.index.get_level_values("date").unique().sort_values()
splits = list(walk_forward_splits(exp_dates, TRAIN_WINDOW, PURGE_GAP))

# Train a final model on each of last 12 windows, collect SHAP
shap_values_pooled = []
feature_names = all_feature_cols
n_shap_windows = min(12, len(splits))

for train_dates, pred_date in splits[-n_shap_windows:]:
    train_mask = panel_exp.index.get_level_values("date").isin(train_dates)
    pred_mask = panel_exp.index.get_level_values("date") == pred_date
    X_tr = panel_exp.loc[train_mask, all_feature_cols].values
    y_tr = panel_exp.loc[train_mask, "fwd_return"].values
    X_pred = panel_exp.loc[pred_mask, all_feature_cols].values

    mdl = lgb.LGBMRegressor(
        n_estimators=200, learning_rate=0.05, num_leaves=31,
        min_child_samples=20, reg_alpha=0.1, reg_lambda=1.0,
        random_state=42, verbosity=-1,
    )
    # Split for early stopping
    n_val = min(12, len(train_dates) // 5)
    fit_end = len(train_dates) - n_val
    fit_mask = panel_exp.index.get_level_values("date").isin(train_dates[:fit_end])
    val_mask = panel_exp.index.get_level_values("date").isin(train_dates[fit_end:])
    X_fit = panel_exp.loc[fit_mask, all_feature_cols].values
    y_fit = panel_exp.loc[fit_mask, "fwd_return"].values
    X_val = panel_exp.loc[val_mask, all_feature_cols].values
    y_val = panel_exp.loc[val_mask, "fwd_return"].values
    mdl.fit(X_fit, y_fit, eval_set=[(X_val, y_val)],
            callbacks=[lgb.early_stopping(50, verbose=False)])

    explainer = shap.TreeExplainer(mdl)
    sv = explainer.shap_values(X_pred)
    shap_values_pooled.append(sv)

We pool SHAP values across the last 12 walk-forward windows rather than using a single snapshot. A single window's SHAP values are unstable on this data — the feature rankings can change substantially from one month to the next. Pooling across 12 windows gives a more robust picture of which features the model consistently relies on.

In [ ]:
shap_all = np.vstack(shap_values_pooled)
mean_abs_shap = np.mean(np.abs(shap_all), axis=0)
shap_ranking = sorted(
    zip(feature_names, mean_abs_shap),
    key=lambda x: x[1], reverse=True,
)

print(f"  SHAP windows pooled: {n_shap_windows}")
print(f"  Top-10 features by mean |SHAP|:")
for rank, (feat, val) in enumerate(shap_ranking[:10], 1):
    origin = "original" if feat in FEATURE_COLS else "engineered"
    print(f"    {rank}. {feat}: {val:.6f} ({origin})")
print()

Volatility dominates the SHAP ranking at 0.00121 — 1.6 times the next feature (rvol_3m at 0.00101). Together, the two volatility features account for about 40% of total SHAP importance. This aligns precisely with the GKX (2020) finding that a small handful of features drive predictions regardless of model complexity. The interaction feature mom_x_vol ranks in the top 5, suggesting that conditional momentum — the idea that momentum's strength varies with volatility regime — captures real cross-sectional structure beyond what the pure linear features provide.

Now the economic interpretations. Every feature that drives a model's predictions should have a story — a reason why it should predict cross-sectional returns.

In [ ]:
interpretations = {
    "momentum_z": "Cross-sectional momentum: stocks with recent outperformance "
                  "tend to continue outperforming — a well-documented anomaly.",
    "reversal_z": "Short-term reversal: extreme recent losers bounce back, "
                  "consistent with microstructure-driven mean reversion.",
    "volatility_z": "Higher idiosyncratic volatility correlates with higher "
                    "expected returns in cross-section (risk compensation).",
    "pb_ratio_z": "Price-to-book captures the value premium — stocks with low "
                  "P/B tend to earn higher returns (Fama-French HML factor).",
    "roe_z": "Quality signal: high-ROE firms have persistent earnings power, "
             "which the market may undervalue.",
    "earnings_yield_z": "Earnings yield (E/P) is the inverse of P/E — captures "
                        "cheap vs. expensive stocks in cross-section.",
    "asset_growth_z": "Firms expanding assets aggressively tend to underperform "
                      "(Cooper, Gulen & Schill 2008).",
    "mom_3m": "Short-horizon momentum captures different frequency dynamics "
              "than 12-month momentum — more sensitive to recent catalysts.",
    "mom_6m": "Intermediate momentum: smoother than 3-month, less reversal-prone "
              "than 12-month. Captures medium-term trends.",
    "mom_12m_1m": "Skip-month momentum: Jegadeesh & Titman's (1993) original "
                  "formulation, removing the reversal-prone most recent month.",
    "rvol_3m": "Realized volatility: higher vol stocks carry a risk premium "
               "but are also noisier — the Ang et al. (2006) low-vol anomaly.",
    "amihud": "Amihud illiquidity: less liquid stocks earn a liquidity premium "
              "(compensation for trading difficulty).",
    "de_ratio": "Leverage signal: higher D/E firms are riskier — may earn "
                "a risk premium or signal financial distress.",
    "profit_margin": "Profitability signal: higher-margin firms have durable "
                     "competitive advantages — quality factor proxy (Novy-Marx 2013).",
    "fcf_yield": "Free cash flow yield: high FCF/market cap indicates firms "
                 "generating cash relative to their valuation — quality proxy.",
    "mom_x_vol": "Interaction: momentum effect may be stronger/weaker in "
                 "high-vol regimes — captures conditional momentum.",
    "val_x_qual": "Interaction: value-quality overlap — cheap + profitable "
                  "stocks may have additive alpha (Asness et al. 2019).",
    "mom_sq": "Non-linear momentum: captures diminishing returns at extreme "
              "momentum — very high momentum stocks may reverse.",
    "abs_reversal": "Absolute reversal: magnitude of recent loss/gain matters "
                    "regardless of sign — captures extreme-move effect.",
}

Each interpretation maps a feature to an economic mechanism. In production, features without economic stories are treated with suspicion — if you cannot explain why a feature should predict returns, you may be fitting noise. This is the domain knowledge that feature engineering at a fund requires: not just knowing how to compute a feature, but knowing why it should work.

In [ ]:
print("── ECONOMIC INTERPRETATION (Top-5) ───────────────────")
for rank, (feat, _) in enumerate(shap_ranking[:5], 1):
    interp = interpretations.get(feat, "No standard interpretation available.")
    print(f"  {rank}. {feat}: {interp}")
print()

The top-5 ranking reveals the model's true dependencies: volatility (risk compensation), 3-month realized vol (a different frequency of the same risk signal), price-to-book (the value premium), Amihud illiquidity (the liquidity premium), and the momentum-volatility interaction. These are exactly the categories that GKX (2020) found dominant across all model types — the features are the alpha, and the model is just the lens.

Finally, the PIT contamination check. We strip out all fundamental features (the four original z-scored fundamentals plus the two new static ratios) and re-run the pipeline on only price-derived, PIT-clean features.

In [ ]:
print("── PIT CONTAMINATION CHECK ───────────────────────────")
pit_clean_cols = [c for c in all_feature_cols
                  if c not in ["pb_ratio_z", "roe_z", "asset_growth_z",
                               "earnings_yield_z", "de_ratio", "profit_margin"]]
pit_all_cols = all_feature_cols

print(f"  PIT-clean features ({len(pit_clean_cols)}): {pit_clean_cols}")
print(f"  All features ({len(pit_all_cols)}): {pit_all_cols}")

We strip out all six fundamental features — four original z-scored fundamentals plus the two static ratios added in this deliverable — leaving only price-derived, volume-derived, and interaction features. The pipeline re-runs on this clean subset, giving us a direct measure of how much the contaminated fundamentals contribute (or detract).

In [ ]:
pit_clean_features = expanded_features[pit_clean_cols].loc[common_idx]
pit_model = lgb.LGBMRegressor(
    n_estimators=200, learning_rate=0.05, num_leaves=31,
    min_child_samples=20, reg_alpha=0.1, reg_lambda=1.0,
    subsample=0.8, subsample_freq=1, colsample_bytree=0.8,
    random_state=42, verbosity=-1,
)
pit_pipeline = AlphaModelPipeline(
    model=pit_model,
    features=pit_clean_features,
    target=expanded_target,
    train_window=TRAIN_WINDOW,
    purge_gap=PURGE_GAP,
    cost_bps=COST_BPS,
    hp_search=True,
    impute=False,
)
pit_ic_df, _ = pit_pipeline.fit_predict()
pit_summary = pit_pipeline.summary()

pit_diff = exp_summary["mean_ic"] - pit_summary["mean_ic"]
print(f"  PIT-clean mean IC: {pit_summary['mean_ic']:.4f}")
print(f"  All features mean IC: {exp_summary['mean_ic']:.4f}")
print(f"  Difference (all - clean): {pit_diff:+.4f}")
if pit_diff > 0.01:
    print(f"  ⚠ PIT contamination likely inflating IC by {pit_diff:.4f}")
else:
    print(f"  Minimal PIT contamination effect")
print()

Here is the counterintuitive punchline: the PIT-clean model (using only price-derived features) produces a *higher* IC than the full model including contaminated fundamentals. The PIT-contaminated features hurt IC by about 0.007. This is the opposite of what you might expect — perfect foreknowledge of fundamentals should help, right? Not on a 174-stock S&P 500 cross-section, where the fundamental rankings are too stable (remember, these are static ratios applied uniformly across all months) to provide useful month-to-month prediction differentiation. The cross-sectional variation in static D/E ratios and profit margins is noise that dilutes the signal from time-varying price features. In production, with quarterly point-in-time fundamentals and a 3,000-stock universe, the story would be different — but on our sandbox data, the message is clear: bad data is worse than no data.

Let us visualize the SHAP importance and the baseline-versus-expanded comparison.

In [ ]:
fig_shap, ax_shap = plt.subplots(figsize=(10, 8))
feature_order = [f for f, _ in shap_ranking]
ordered_indices = [feature_names.index(f) for f in feature_order]
shap_ordered = shap_all[:, ordered_indices]

# Color: original features blue, engineered orange
colors_bar = ["#1f77b4" if f in FEATURE_COLS else "#ff7f0e"
              for f in feature_order]

ax_shap.barh(range(len(feature_order)), mean_abs_shap[ordered_indices],
             color=colors_bar)
ax_shap.set_yticks(range(len(feature_order)))
ax_shap.set_yticklabels(feature_order)
ax_shap.invert_yaxis()
ax_shap.set_xlabel("Mean |SHAP value|")
ax_shap.set_title("Feature Importance: Expanded Model (blue=original, orange=engineered)")
plt.tight_layout()
plt.show()

The horizontal bar chart reveals the importance hierarchy at a glance. The blue bars (original features) and orange bars (engineered features) are interspersed — volatility_z (original) leads, rvol_3m (engineered) follows closely. The engineered interaction feature mom_x_vol earns its place in the top 5, validating the domain intuition that conditional momentum captures real structure.

In [ ]:
fig_comp, ax_comp = plt.subplots(figsize=(8, 5))
labels = ["Baseline\n(7 features)", "Expanded\n({0} features)".format(n_features)]
means = [base_summary["mean_ic"], exp_summary["mean_ic"]]
stds = [base_summary["std_ic"] / np.sqrt(base_summary["n_oos_months"]),
        exp_summary["std_ic"] / np.sqrt(exp_summary["n_oos_months"])]
bars = ax_comp.bar(labels, means, yerr=stds, capsize=5,
                   color=["#1f77b4", "#ff7f0e"], width=0.5)
ax_comp.axhline(0, color="black", linewidth=0.5)
ax_comp.set_ylabel("Mean OOS IC")
ax_comp.set_title(f"Feature Expansion: IC Change = {ic_change:+.4f} (p={p_paired:.3f})")
plt.tight_layout()
plt.show()

The error bars overlap — the IC degradation from feature expansion is not statistically significant (p = 0.13). But the direction is clear and consistent across multiple runs. At a fund, this chart would end the conversation about adding these features to the production model at this universe scale. You would file it under "revisit when we have 3,000 stocks."

The core lesson from this deliverable: more features is not always better. Feature expansion that improves performance on a 3,000-stock, 94-feature universe can actively degrade performance on a 174-stock, 18-feature subset. The production result requires both a larger feature set and a larger cross-section to materialize. This is one of the most important lessons for ML engineers entering finance — the relationship between feature count and signal quality is not monotonic, and it depends critically on the scale of your data.

---

## Deliverable 3: The Model Comparison Report

Your fund's CIO walks into your office and asks the question that defines a quant team's technology stack: "Should we switch from gradient boosting to neural networks?" This is not a casual question. Rewriting a production pipeline costs engineering months, introduces new failure modes (GPU infrastructure, training instability, hyperparameter sensitivity), and requires revalidating every downstream system. The CIO needs a rigorous, honest answer — not hype about "deep learning" and not reflexive conservatism about "if it ain't broke."

Your job is to produce that answer. You will run four model families — OLS, Ridge, LightGBM, and a feedforward neural network — through the same pipeline on the same expanded feature set. You will evaluate on the metrics that actually matter (IC, ICIR, Sharpe, turnover, net Sharpe), compute pairwise statistical tests, apply the deflated Sharpe ratio for multiple testing, and write a structured recommendation.

The teaser: the model ranking on our data is the *inverse* of what GKX (2020) found on production data. That inversion — and the reason behind it — is the most important finding in this homework.

### Student Workspace

In [ ]:
# TODO: Load D2's expanded feature matrix
# TODO: Define all 4 model configurations (OLS, Ridge, LightGBM, NN)
# TODO: Build a sklearn-compatible NN wrapper with fit/predict methods
# TODO: Run each model through the AlphaModelPipeline

After running all models, analyze and compare the results statistically. The summary table should contain every metric from `pipeline.summary()`, and the pairwise paired t-tests should compare adjacent complexity levels (OLS vs Ridge, Ridge vs GBM, GBM vs NN) to determine whether the IC differences are significant.

In [ ]:
# TODO: Construct the summary comparison table
# TODO: Compute pairwise paired t-tests between adjacent complexity levels
# TODO: Compute deflated Sharpe ratio for the best model

Finally, contextualize the results with an honest sandbox-vs-production analysis. Document the four gaps between our sandbox and a production deployment (feature count, universe size, PIT contamination, survivorship bias), write the CIO recommendation, and produce the three-panel comparison chart.

In [ ]:
# TODO: Write the sandbox-vs-production analysis (features, universe, PIT, survivorship)
# TODO: Write the CIO recommendation
# TODO: Visualize: 3-panel chart (IC, ICIR, Sharpe)

---

## ━━━ SOLUTION: Deliverable 3 ━━━

We start by loading D2's expanded feature matrix and aligning it with the target variable. All four models will operate on the exact same data — the only thing that changes is the model.

In [ ]:
expanded_features = pd.read_parquet(CACHE_DIR / "expanded_features.parquet")
panel = load_monthly_panel()
target = panel["fwd_return"]

# Align
common_idx = expanded_features.index.intersection(target.index)
expanded_features = expanded_features.loc[common_idx]
target = target.loc[common_idx]
feature_cols = list(expanded_features.columns)

print("── DATA ──────────────────────────────────────────────")
print(f"  Expanded features: {len(feature_cols)} columns")
print(f"  Shape: {expanded_features.shape}")
print(f"  Target: {len(target)} observations")
print()

Before running the comparison, we need a neural network that speaks sklearn. The AlphaModelPipeline expects `fit()` and `predict()` methods — the standard sklearn protocol. We build a wrapper around a two-layer PyTorch network that provides this interface.

In [ ]:
class AlphaNet(nn.Module):
    """Two-layer feedforward for cross-sectional return prediction."""

    def __init__(self, n_features, hidden=32, dropout=0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_features, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, hidden // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden // 2, 1),
        )

    def forward(self, x):
        return self.net(x).squeeze(-1)

The architecture — 32 neurons in the first hidden layer, 16 in the second, ReLU activation, 30% dropout — matches the GKX NN1/NN2 specification. This is deliberately small. On ~10,000 training observations with 18 features, a larger network would overfit catastrophically. The dropout rate of 0.3 is the regularization mechanism, serving the same role as early stopping in LightGBM. In production, firms like Man Group AHL use similar shallow networks for tabular cross-sectional data; deep architectures (5+ layers) are reserved for unstructured inputs like text and images.

In [ ]:
class NNRegressor:
    """Sklearn-compatible wrapper for PyTorch NN."""

    def __init__(self, n_features, hidden=32, dropout=0.3, lr=1e-3,
                 epochs=30, batch_size=256, device="cpu"):
        self.n_features = n_features
        self.hidden = hidden
        self.dropout = dropout
        self.lr = lr
        self.epochs = epochs
        self.batch_size = batch_size
        self.device = device
        self.model_ = None

The wrapper stores hyperparameters in its constructor but delays model creation until `fit()` — a fresh network is initialized for every walk-forward window. This prevents weight leakage across windows, which would be a subtle but serious form of temporal contamination.

In [ ]:
def _nn_fit(self, X, y, **kwargs):
    self.model_ = AlphaNet(self.n_features, self.hidden, self.dropout)
    X = X.copy()
    y = y.copy()
    # Use last 20% as validation for early stopping
    n_val = max(1, len(X) // 5)
    X_tr, y_tr = X[:-n_val], y[:-n_val]
    X_val, y_val = X[-n_val:], y[-n_val:]
    # Impute NaN for NN (can't handle natively)
    for j in range(X_tr.shape[1]):
        med = np.nanmedian(X_tr[:, j])
        X_tr[np.isnan(X_tr[:, j]), j] = med if np.isfinite(med) else 0.0
        X_val[np.isnan(X_val[:, j]), j] = med if np.isfinite(med) else 0.0
    fit_nn(
        self.model_, X_tr, y_tr,
        x_val=X_val, y_val=y_val,
        epochs=self.epochs, lr=self.lr,
        batch_size=self.batch_size,
        patience=10, device=self.device,
    )
    return self

NNRegressor.fit = _nn_fit

The NaN imputation happens inside `fit()` using training set medians — the same pattern as the pipeline's imputation, but necessary here because PyTorch tensors cannot contain NaN. The validation split for early stopping uses the last 20% of the training window (temporal, never shuffled). The `fit_nn` function from `shared/dl_training.py` handles the training loop, gradient clipping, and learning rate scheduling.

In [ ]:
def _nn_predict(self, X):
    # Impute NaN
    X = X.copy()
    for j in range(X.shape[1]):
        med = np.nanmedian(X[:, j])
        X[np.isnan(X[:, j]), j] = med if np.isfinite(med) else 0.0
    return predict_nn(self.model_, X, device=self.device)

NNRegressor.predict = _nn_predict

The `predict()` method handles its own NaN imputation using column medians from the prediction cross-section. This is a pragmatic choice — ideally you would store the training medians from `fit()` and apply them at prediction time, but for walk-forward evaluation where each window retrains from scratch, the difference is negligible. The final piece of the sklearn protocol is the parameter introspection methods that `clone()` needs.

In [ ]:
def _nn_get_params(self, deep=True):
    return dict(n_features=self.n_features, hidden=self.hidden,
                dropout=self.dropout, lr=self.lr, epochs=self.epochs,
                batch_size=self.batch_size, device=self.device)

def _nn_set_params(self, **params):
    for k, v in params.items():
        setattr(self, k, v)
    return self

def _nn_sklearn_clone(self):
    return NNRegressor(**self.get_params())

NNRegressor.get_params = _nn_get_params
NNRegressor.set_params = _nn_set_params
NNRegressor.__sklearn_clone__ = _nn_sklearn_clone

The `get_params`, `set_params`, and `__sklearn_clone__` methods complete the sklearn protocol. Without these, the pipeline's `clone(self.model)` call would fail. This is the kind of plumbing that is invisible in research notebooks but essential for production compatibility. At a fund, your NNRegressor must be a drop-in replacement for any sklearn model — if it is not, the infrastructure team will reject it.

Now the four model configurations. Each one is defined with its specific settings.

In [ ]:
n_feat = len(feature_cols)

model_configs = {
    "OLS": {
        "model": LinearRegression(),
        "hp_search": False,
        "impute": True,
    },
    "Ridge": {
        "model": Ridge(alpha=1.0),
        "hp_search": False,
        "impute": True,
    },
    "LightGBM": {
        "model": lgb.LGBMRegressor(
            n_estimators=200, learning_rate=0.05, num_leaves=31,
            min_child_samples=20, reg_alpha=0.1, reg_lambda=1.0,
            subsample=0.8, subsample_freq=1, colsample_bytree=0.8,
            random_state=42, verbosity=-1,
        ),
        "hp_search": True,
        "impute": False,
    },
    "NN": {
        "model": NNRegressor(
            n_features=n_feat, hidden=32, dropout=0.3,
            lr=1e-3, epochs=30, batch_size=256, device="cpu",
        ),
        "hp_search": False,
        "impute": False,  # NNRegressor handles NaN internally
    },
}

OLS and Ridge use `impute=True` because they cannot handle NaN. LightGBM handles it natively. The NN wrapper imputes inside its own `fit()` and `predict()` methods, so the pipeline's imputation is off. This design — each model managing its own NaN strategy — mirrors production systems where different model families coexist in the same evaluation framework.

Time to run all four models. This is the most compute-intensive step in the homework — each model runs ~68 walk-forward windows, and the NN has the additional overhead of PyTorch training at each window.

In [ ]:
results = {}
ic_series_all = {}

for name, config in model_configs.items():
    print(f"── MODEL: {name} ─────────────────────────────────────")
    pipeline = AlphaModelPipeline(
        model=config["model"],
        features=expanded_features,
        target=target,
        train_window=TRAIN_WINDOW,
        purge_gap=PURGE_GAP,
        cost_bps=COST_BPS,
        hp_search=config["hp_search"],
        impute=config["impute"],
    )
    ic_df, pred_df = pipeline.fit_predict()
    summary = pipeline.summary()
    results[name] = summary
    ic_series_all[name] = ic_df["ic_pearson"].dropna()
    print(f"  mean IC: {summary['mean_ic']:.4f}, "
          f"ICIR: {summary['icir']:.4f}, "
          f"Sharpe: {summary['sharpe_gross']:.4f}")
    print()

The same `AlphaModelPipeline` class — built in Deliverable 1 — handles all four models without modification. That is the point of building a reusable pipeline: one piece of infrastructure, any model family. A quantitative researcher at Millennium or Citadel uses exactly this pattern — the evaluation framework is fixed, and models are swapped in and out as experiments.

Now the comprehensive comparison table.

In [ ]:
print("── MODEL COMPARISON SUMMARY TABLE ────────────────────")
header = (f"{'Model':<10} {'IC':>7} {'σ(IC)':>7} {'ICIR':>7} "
          f"{'pct+':>7} {'t-stat':>7} {'p-val':>7} {'rIC':>7} "
          f"{'SR_g':>7} {'SR_n':>7} {'TO':>7} {'MDD':>7}")
print(f"  {header}")
print(f"  {'─' * len(header)}")
for name in model_configs:
    s = results[name]
    row = (f"  {name:<10} "
           f"{s['mean_ic']:>7.4f} {s['std_ic']:>7.4f} {s['icir']:>7.4f} "
           f"{s['pct_positive']:>7.4f} {s['t_stat']:>7.4f} {s['p_value']:>7.4f} "
           f"{s['mean_rank_ic']:>7.4f} "
           f"{s['sharpe_gross']:>7.4f} {s['sharpe_net']:>7.4f} "
           f"{s['mean_turnover']:>7.4f} {s['max_drawdown']:>7.4f}")
    print(row)
print()

Study this table carefully. The model ranking is OLS/Ridge > LightGBM > NN — the *inverse* of the GKX (2020) production ranking (NN >= GBRT > Ridge > OLS). OLS and Ridge both achieve IC around 0.044, LightGBM at 0.034, and the NN at only 0.018. But here is the critical finding: *no individual model's IC reaches 5% significance* on the expanded feature set (all p-values above 0.11). The entire comparison operates on signals that cannot be statistically distinguished from zero.

The Sharpe comparison is even more stark: Ridge at approximately 0.83 versus NN at 0.27 — a 3x difference. This is not because neural networks are bad models; it is because on 18 correlated features and 174 S&P 500 stocks, the additional flexibility of a neural network finds noise instead of signal. The linear models cannot overfit as badly, so they produce more stable (albeit equally weak) predictions.

Now the pairwise statistical tests.

In [ ]:
pairs = [("Ridge", "OLS"), ("LightGBM", "Ridge"), ("NN", "LightGBM")]

print("── PAIRWISE PAIRED T-TESTS ───────────────────────────")
for m1, m2 in pairs:
    ic1 = ic_series_all[m1].values
    ic2 = ic_series_all[m2].values
    n = min(len(ic1), len(ic2))
    diff = ic1[:n] - ic2[:n]
    if n >= 2 and diff.std() > 0:
        t = diff.mean() / (diff.std() / np.sqrt(n))
        p = 2 * (1 - stats.t.cdf(abs(t), df=n - 1))
    else:
        t, p = np.nan, np.nan
    sig = "YES" if (np.isfinite(p) and p < 0.05) else "no"
    print(f"  {m1} vs {m2}: t={t:+.4f}, p={p:.4f} [{sig}]")
print()

No pairwise comparison reaches 5% significance (all p-values above 0.26). The IC spread of 0.025 across all four models is within the noise of monthly IC estimation (standard error approximately 0.02). This means we cannot statistically distinguish any model from any other — the comparison is uninformative at this scale. This is itself the most important finding: with 18 features and 174 stocks, model choice does not matter.

The deflated Sharpe ratio adjusts for multiple testing — the fact that we tried 4 models and picked the best one.

In [ ]:
n_trials = len(model_configs)
best_model = max(results, key=lambda m: results[m]["sharpe_gross"])
best_sharpe = results[best_model]["sharpe_gross"]
best_n_oos = results[best_model]["n_oos_months"]

# Compute return skewness and kurtosis for deflated SR
best_ic_series = ic_series_all[best_model]
skew_val = float(stats.skew(best_ic_series.values))
kurt_val = float(stats.kurtosis(best_ic_series.values))

dsr = deflated_sharpe_ratio(
    best_sharpe, n_trials, best_n_oos,
    skew=skew_val, excess_kurt=kurt_val,
)

The DSR computation uses the IC series' skewness and excess kurtosis because return distributions in cross-sectional equity strategies are non-Gaussian. Ignoring these higher moments would overstate the statistical significance of the Sharpe ratio — a common mistake in backtesting research. Now let us see whether the best model survives the adjustment.

In [ ]:
print("── MULTIPLE TESTING AWARENESS ────────────────────────")
print(f"  Models compared: {n_trials}")
print(f"  Best model: {best_model} (gross Sharpe = {best_sharpe:.4f})")
print(f"  Deflated Sharpe Ratio probability: {dsr:.4f}")
if dsr > 0.95:
    print(f"  Best SR survives deflation (DSR > 0.95)")
else:
    print(f"  Best SR does NOT survive deflation (DSR < 0.95)")
    print(f"  With only {n_trials} trials, this mainly indicates a low SR")
print()

The deflated Sharpe ratio (Bailey and Lopez de Prado, 2014) adjusts for the selection bias of picking the best model from multiple trials. With only 4 trials, the adjustment is small — the main driver of the DSR result is the absolute level of the Sharpe ratio, not the number of trials. A DSR above 0.95 means the best model's Sharpe is robust to the multiple-testing adjustment. At a fund that tests hundreds of signals, the DSR correction would be much more severe.

Now the sandbox-vs-production analysis — the section that separates honest research from marketing.

In [ ]:
print("── SANDBOX vs PRODUCTION ─────────────────────────────")
print("  (a) FEATURE GAP:")
print(f"      Sandbox: {len(feature_cols)} features")
print(f"      Production (GKX 2020): 94 firm characteristics")
print(f"      → More features capture richer cross-sectional variation.")
print(f"         GKX found ~50% OOS R² improvement going from 10 to 94 features.")
print()
print("  (b) UNIVERSE GAP:")
print(f"      Sandbox: ~174 S&P 500 stocks")
print(f"      Production (GKX 2020): ~3,000 CRSP stocks")
print(f"      → Larger cross-section provides more power to distinguish models")
print(f"         and includes less-efficient small/mid-caps where alpha is larger.")
print()

The feature and universe gaps compound: fewer features on fewer stocks means the non-linear models have both less information to work with and less statistical power to exploit it. The next two gaps — data quality issues — affect absolute performance more than relative model rankings.

In [ ]:
print("  (c) PIT CONTAMINATION:")
print(f"      Sandbox: 4 original + 2 new fundamental features use static ratios")
print(f"      Production: quarterly filings with proper point-in-time handling")
print(f"      → Our fundamental features have look-ahead bias. IC may be inflated")
print(f"         for models that rely on fundamentals. Price-derived features are clean.")
print()
print("  (d) SURVIVORSHIP BIAS:")
print(f"      Sandbox: current S&P 500 members applied retroactively")
print(f"      Production: point-in-time index constituents from CRSP")
print(f"      → Survivorship bias inflates returns by ~1-2% annualized (Brown 1992).")
print(f"         All models are equally affected, so relative rankings may be valid.")
print()

Each gap has a directional impact. The feature gap means our models are starved — 18 features versus the 94 that GKX used. The universe gap means our cross-section is too small for non-linear models to find the complex interactions they are designed to exploit. The PIT contamination means our fundamental features are unreliable. And the survivorship bias inflates all returns equally, making the absolute Sharpe numbers unreliable while leaving relative rankings approximately valid.

At a fund, this analysis would be the first page of the model comparison memo — before the results, before the tables. The CIO needs to know what the results can and cannot tell them before they see a single number.

Now the recommendation — the punchline.

In [ ]:
model_names = list(results.keys())
mean_ics = [results[m]["mean_ic"] for m in model_names]
best_ic_model = model_names[np.argmax(mean_ics)]
worst_ic_model = model_names[np.argmin(mean_ics)]
ic_range = max(mean_ics) - min(mean_ics)

print("── CIO RECOMMENDATION ────────────────────────────────")
print(f"  RECOMMENDATION: The model comparison does NOT support switching")
print(f"  from gradient boosted trees to neural networks on this data.")
print()
print(f"  The IC spread across all {n_trials} models is {ic_range:.4f} — within")
print(f"  the noise of monthly IC estimation (SE ~0.02). No pairwise comparison")
print(f"  reaches 5% significance. The best model ({best_ic_model}) outperforms")
print(f"  the worst ({worst_ic_model}) by {ic_range:.4f} in IC, but this is not")
print(f"  statistically distinguishable from zero.")
print()
print(f"  The case for neural networks requires: (a) unstructured data inputs")
print(f"  (text, images) that GBM cannot natively handle, (b) a substantially")
print(f"  larger feature set (94+ characteristics per GKX 2020), or (c) a larger")
print(f"  cross-section (~3,000 stocks) where non-linear interactions have more")
print(f"  statistical power. On 174 S&P 500 stocks with 18 features, model")
print(f"  choice is dominated by noise, not signal.")
print()

This recommendation is honest, specific, and actionable. It does not say "neural networks are bad" — it says "neural networks are not justified on *this* data, at *this* scale." The conditions under which neural networks would be justified are explicit: unstructured data, larger feature sets, larger cross-sections. A CIO reading this knows exactly what to invest in (more data, broader universe) rather than what model to buy.

Finally, the net Sharpe comparison and visualization.

In [ ]:
print("── NET SHARPE COMPARISON (10 bps one-way) ────────────")
for name in model_configs:
    s = results[name]
    print(f"  {name:<10}: gross={s['sharpe_gross']:.4f}, "
          f"net={s['sharpe_net']:.4f}, "
          f"turnover={s['mean_turnover']:.4f}")
any_positive_net = any(results[m]["sharpe_net"] > 0 for m in model_configs)
if any_positive_net:
    print(f"  At least one model shows positive net Sharpe at 10 bps")
else:
    print(f"  All models show negative net Sharpe — sandbox limitations dominate")
print()

The net Sharpe comparison at 10 bps one-way cost is the metric that matters for production viability. At institutional S&P 500 execution costs (5-15 bps per Frazzini, Israel & Moskowitz, 2018), the linear models' signals survive — but only barely. The NN's net Sharpe of approximately 0.20 is marginal at best. In production, the portfolio construction step (Week 6) would add turnover constraints that reduce costs but also reduce signal utilization — a trade-off that is the daily bread of portfolio management.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
model_labels = list(model_configs.keys())
x = np.arange(len(model_labels))
width = 0.6

# Panel A: Mean IC
ics = [results[m]["mean_ic"] for m in model_labels]
ses = [results[m]["std_ic"] / np.sqrt(results[m]["n_oos_months"])
       for m in model_labels]
colors = plt.cm.Set2(np.linspace(0, 0.8, len(model_labels)))
axes[0].bar(x, ics, width, yerr=ses, capsize=4, color=colors)
axes[0].set_xticks(x)
axes[0].set_xticklabels(model_labels)
axes[0].set_ylabel("Mean OOS IC")
axes[0].set_title("Mean IC (±SE)")
axes[0].axhline(0, color="black", linewidth=0.5)

# Panel B: ICIR
icirs = [results[m]["icir"] for m in model_labels]
axes[1].bar(x, icirs, width, color=colors)
axes[1].set_xticks(x)
axes[1].set_xticklabels(model_labels)
axes[1].set_ylabel("ICIR")
axes[1].set_title("Information Coefficient IR")
axes[1].axhline(0, color="black", linewidth=0.5)

# Panel C: Gross vs Net Sharpe
gross = [results[m]["sharpe_gross"] for m in model_labels]
net = [results[m]["sharpe_net"] for m in model_labels]
axes[2].bar(x - 0.15, gross, 0.3, label="Gross", color="#1f77b4")
axes[2].bar(x + 0.15, net, 0.3, label=f"Net ({COST_BPS}bps)", color="#ff7f0e")
axes[2].set_xticks(x)
axes[2].set_xticklabels(model_labels)
axes[2].set_ylabel("Ann. Sharpe")
axes[2].set_title("Sharpe Ratio: Gross vs Net")
axes[2].legend()
axes[2].axhline(0, color="black", linewidth=0.5)

plt.tight_layout()
plt.show()

The three-panel chart tells the complete story at a glance. Panel A shows the IC error bars overlapping heavily — no model is statistically distinguishable from another. Panel B shows ICIR telling a similar story, though LightGBM and the linear models are tightly clustered while the NN falls behind. Panel C shows the economic bottom line: gross Sharpe is positive for all models, but the gap between gross and net bars — the cost of trading — is substantial. Ridge leads in both gross and net Sharpe, not because it is a "better" model, but because its linear predictions produce more stable portfolio rankings month to month, resulting in lower turnover.

---

## What You Built

The pipeline you constructed over these three deliverables — `AlphaModelPipeline`, the feature expansion, and the model comparison — is a simplified version of what a quantitative researcher at a systematic fund operates every day. The core architecture is the same: walk-forward temporal splitting, signal evaluation via IC, portfolio construction via quantile sorts, and cost-adjusted performance measurement.

The surprising findings tell a story about scale dependence. On our 174-stock, 18-feature sandbox, feature expansion degraded IC by 0.020 (baseline 0.055 vs. expanded 0.034), the model ranking inverted relative to published results (linear > GBM > NN), and no individual model produced a statistically significant signal on the expanded feature set. These results are not failures — they are the honest outcomes of running production-grade methodology at sandbox scale. The GKX (2020) result, where neural networks marginally outperform gradient boosting on 94 features and 3,000 stocks, requires exactly the scale our sandbox lacks. Understanding *when* complexity helps — and when it hurts — is the meta-lesson that separates a practitioner from a Kaggle competitor.

What is still missing? Our pipeline uses a simple quantile sort for portfolio construction — Week 6 will replace this with mean-variance optimization and turnover constraints. Our backtesting has no combinatorial purged cross-validation — Week 5 will add that. Our transaction cost model is a flat per-trade assumption — production models use market-impact functions that scale with order size. And our fundamental features have full look-ahead bias — production systems use point-in-time databases like Compustat's Xpressfeed or Refinitiv's DataScope.

## Career Connection

The `AlphaModelPipeline` you built is a pedagogically transparent version of what Microsoft Qlib and similar open-source frameworks provide for production alpha research. A quantitative researcher at a multi-manager pod shop like Millennium or Citadel owns a pipeline exactly like this — their performance is measured by net Sharpe after costs, which is literally the number your `summary()` method returns. The feature engineering process in Deliverable 2 is the daily work of the feature engineering team at a systematic fund, which often has more headcount than the modeling team. And the CIO recommendation in Deliverable 3 is the kind of memo that junior quants write when their team evaluates new technology — honest, data-driven, and specific about what would change the answer. The skills you exercised here — temporal integrity, domain-motivated feature engineering, honest model comparison — are the ones that get you past the first round of interviews.

## Bridge to Next Week

You now have a cross-sectional alpha model that produces signals and constructs portfolios. But how do you know the results are real? Week 5 takes this pipeline and stress-tests it — purged cross-validation, combinatorial purged CV, the deflated Sharpe ratio (which you briefly touched in D3), and transaction cost modeling that goes far beyond our simple spread estimate. The question shifts from "can I build a signal?" to "should I believe it?"